In [1]:
import os, datetime
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()

PH_TZ = datetime.timezone(datetime.timedelta(hours=8))
mongo_uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")

client = MongoClient(mongo_uri, tz_aware=True)
db = client[db_name]
col = db["pump_inventories"]

def get_pump_inventories_by_date(date_str: str):
    # Parse YYYY-MM-DD as PH local midnight
    y, m, d = map(int, date_str.split("-"))
    dt_ph = datetime.datetime(y, m, d, 0, 0, 0, tzinfo=PH_TZ)
    start_utc = dt_ph.astimezone(datetime.timezone.utc)
    end_utc = (dt_ph + datetime.timedelta(days=1)).astimezone(datetime.timezone.utc)

    docs = list(col.find(
        {
            "date": {"$gte": start_utc, "$lt": end_utc},
            "location": "loboc"
        },
        {"_id": 0}
    ))
    return docs

# Example usage
results = get_pump_inventories_by_date("2025-09-28")
for r in results:
    print(r)


{'id': 'e85f1091-977d-42c0-a415-9688f220a586', 'created_at': datetime.datetime(2025, 9, 27, 18, 40, 25, 678000, tzinfo=<bson.tz_util.FixedOffset object at 0x000001582CEF7A10>), 'location': 'loboc', 'date': datetime.datetime(2025, 9, 27, 16, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x000001582CEF7A10>), 'dispenser_name': 'dispenser-three', 'pump_name': 'regular', 'product': 'regular', 'unit': 'liter', 'price': Decimal128('62.50'), 'beginning_inventory': Decimal128('84.00'), 'calibration': Decimal128('30.00'), 'po': Decimal128('1009.00'), 'cash': Decimal128('14812.50'), 'ending_inventory': Decimal128('886.00'), 'starting_liter_meter': Decimal128('34101.00'), 'ending_liter_meter': Decimal128('34338.00')}
{'id': 'a37eddc1-46ea-47e1-86cd-21702a593d2c', 'created_at': datetime.datetime(2025, 9, 27, 18, 40, 25, 678000, tzinfo=<bson.tz_util.FixedOffset object at 0x000001582CEF7A10>), 'location': 'loboc', 'date': datetime.datetime(2025, 9, 27, 16, 0, tzinfo=<bson.tz_util.FixedOffset object 